In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
import os
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results=2)

In [4]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

In [5]:
model = ChatOpenAI(model='gpt-3.5-turbo')


In [6]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [7]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()
# from langgraph.checkpoint.sqlite import SqliteSaver
# memory = SqliteSaver.from_conn_string(":memory:")

In [8]:
class Agent:
    def __init__(self, model, tools, checkpointer, system='' ):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm",
                                    self.action_exists ,
                                    {True:"action", False:END})
        graph.add_edge("action" ,"llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)
    def action_exists(self, state:AgentState):
        result = state['messages'][-1]
        return(len(result.tool_calls)>0)
    
    def call_openai(self, state:AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)]+messages
        message = self.model.invoke(messages)
        return{"messages":[message]}
    def take_action(self, state:AgentState):
        tool_calls  = state["messages"][-1].tool_calls
        results = []
        for t in tool_calls :
            print (f'invoking tool {t}')
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print ("back to the model!")
        return {"messages": results}
        

In [9]:
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [10]:
messages = [HumanMessage(content="What is the weather in chennai, India")]

In [11]:
thread={"configurable": {"thread_id":"1"}}


In [12]:
for event in abot.graph.stream({"messages":messages}, thread):
    for v in event.values():
        print (v['messages'])
    

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_K8K1hfEXCrvhNTdypQgzguFq', 'function': {'arguments': '{"query":"weather in Chennai, India"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 155, 'total_tokens': 178, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-39acddf8-43c6-48c6-af03-8c9dd7195a77-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Chennai, India'}, 'id': 'call_K8K1hfEXCrvhNTdypQgzguFq', 'type': 'tool_call'}], usage_metadata={'input_tokens': 155, 'output_tokens': 23, 'total_tokens': 178, 'input_token_details': {'audio': 0, 'cache_re

In [13]:
messages = [HumanMessage(content="how about la?")]
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print (v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zaOb8U7VAnECvV7XoS4Xkp14', 'function': {'arguments': '{"query":"weather in Los Angeles, California"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 821, 'total_tokens': 845, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-31b768ba-6bb5-471d-b784-f1bb2c785139-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles, California'}, 'id': 'call_zaOb8U7VAnECvV7XoS4Xkp14', 'type': 'tool_call'}], usage_metadata={'input_tokens': 821, 'output_tokens': 24, 'total_tokens': 845, 'input_token_details': {'au

In [14]:
messages = [HumanMessage(content="how about portland?")]
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print (v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TyqUjJjMFEENbrxrotcSOG0N', 'function': {'arguments': '{"query":"weather in Portland, Oregon"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 1774, 'total_tokens': 1797, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2ba85b42-4130-4f9f-89bf-52333b85c584-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Portland, Oregon'}, 'id': 'call_TyqUjJjMFEENbrxrotcSOG0N', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1774, 'output_tokens': 23, 'total_tokens': 1797, 'input_token_details': {'audio': 0, '

In [15]:
messages = [HumanMessage(content="how about NY?")]
thread={"configurable": {"thread_id":"2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print (v['messages'])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3o5AcEUxA89WLVfZKKE3iHIA', 'function': {'arguments': '{"query":"New York news"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 150, 'total_tokens': 171, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c4013c13-bec8-40a9-a49b-801f6b6cd3f4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'New York news'}, 'id': 'call_3o5AcEUxA89WLVfZKKE3iHIA', 'type': 'tool_call'}], usage_metadata={'input_tokens': 150, 'output_tokens': 21, 'total_tokens': 171, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_d

In [16]:
# from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
# memory = AsyncSqliteSaver.from_conn_string(":memory:")
abot = Agent(model, [tool],system=prompt, checkpointer=memory)

In [17]:
messages = [HumanMessage(content="What is the weather in SF?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="")

invoking tool {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_dqZ3W77XAPY95iI1r83GWlQj', 'type': 'tool_call'}
back to the model!
The current weather in San Francisco is partly cloudy with a temperature of 48.9°F (9.4°C). The wind speed is 21.6 km/h from the west, and the humidity is at 66%.